In [38]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import pyLDAvis.sklearn

import sys
import os
import logging
import re
import csv
import json
import smart_open
import codecs
import nltk
import time
import pickle
import numpy as np
from collections import OrderedDict

from tweets_text_processor import TextProcessor

In [39]:
def load_model(model_file, tweets):
    pipeline = pickle.load(open(model_file, "rb"))
    model = pipeline.transform(tweets)
    lda = pipeline.named_steps['lda']
    vect = pipeline.named_steps['vect']
    return [pipeline, model, lda, vect]

In [40]:
def get_topic_word_dist(topics, feature_names, num_words=100):
    word_dists = []  # [{word1: weight1, word2: weight2, word3: weight3....}, {word1, weight1, word2: weight2....}...]
    for topic_idx, topic in enumerate(topics):
        feature_list = OrderedDict()  # {word1: weight, word2: weight ...}
        for i in topic.argsort()[:-num_words - 1:-1]:
            feature = feature_names[i]
            weight = round(topic[i], 2)
            feature_list[feature] = weight

        word_dists.append(feature_list)
    return word_dists

In [41]:
def get_topic_dist(components, feature_names, model):
    average = np.average(np.array(model), axis=0)
    top_topics = average.argsort()[::-1]
    top_component = [components[i] for i in top_topics]
    top_topics_words = get_topic_word_dist(top_component, feature_names)

    top_topics = np.array(list(map(lambda x:x+1, top_topics)))
    result = {"avg": average, "top_topics": top_topics, "top_component": top_component, "top_topics_words":
              top_topics_words}
    return result

In [42]:
def consine_similarirty(vec1, vec2):
    numerator = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)

    return numerator / (norm_vec1 * norm_vec2)

In [43]:
def compare_topics(pipeline, model, num_bf_doc):
    # TODO find the right threshold
    THRESDHOLD = 20

    components = pipeline._final_estimator.components_
    feature_names = pipeline.named_steps['vect'].get_feature_names()

    # overall topic distributions
    model_t = model
    result_t = get_topic_dist(components, feature_names, model_t)

    model_bf = model[:num_bf_doc]
    result_bf = get_topic_dist(components, feature_names, model_bf)

    top_topics_bf = result_bf['top_topics'][:THRESDHOLD]

    model_af = model[num_bf_doc:]
    result_af = get_topic_dist(components, feature_names, model_af)
    top_topics_af = result_af['top_topics'][:THRESDHOLD]

    print("Topic Distribution Similarity", consine_similarirty(result_bf['avg'], result_af['avg']))
    print("Common topic", list(top_topics_bf | top_topics_af))
    unique_topic = list(set(top_topics_bf).symmetric_difference(set(top_topics_af)))
    print("Unique Topic", unique_topic)
    bf_unique = list(filter(lambda x: x in unique_topic, top_topics_bf))
    af_unique = list(filter(lambda x: x in unique_topic, top_topics_af))
    bf_unique_diff = [str(result_bf["avg"][x-1])for x in bf_unique]
    af_unique_diff = [str(result_af["avg"][x-1])for x in af_unique]
    print("BF unique topic", bf_unique, bf_unique_diff)
    print("AF unique topic", af_unique, af_unique_diff)

    print("\n\n")
    # TODO print unique topics for period

    return result_t, result_bf, result_af

In [44]:
def show_topic_dist(result, period, num_topics=20):
    print("{} topic distribution: \n".format(period))

    top_topics = result["top_topics"]
    print("top_topics: \n", ' | '.join([str(top_topics[i]) + " - " + str(result["avg"][top_topics[i]-1]) for i in range(num_topics)]))
    for i, words in enumerate([', '.join(list(dist.keys())[:15]) for dist in result["top_topics_words"][:num_topics]]):
        print("topic " + str(top_topics[i]) + " top words: \n", words)
    print("\n\n")

In [45]:
def show_topics(result_t, result_bf, result_af):
    # overall topic distributions
    show_topic_dist(result_t, "overall")

    show_topic_dist(result_bf, "before")

    show_topic_dist(result_af, "after")


In [46]:
def extract_tweet_with_period(incident, input_paths, output_path, metadata_path):
    """
    extract tweets before and after disaster start
    input_paths are two paths, geotagged and timeline folders
    """
    file_bf = os.path.join(output_path, "{}_tweets_bf.csv".format(incident))
    file_af = os.path.join(output_path, "{}_tweets_af.csv".format(incident))

    if os.path.isfile(file_bf) and os.path.isfile(file_af):
        logging.debug("Tweets already extracted")
        corpus = []
        files = [file_bf, file_af]
        for file in files:
            with open(file,'r') as file:
                lines = file.readlines()
                tweets = lines[1:]
                corpus.append(tweets)

            logging.debug(tweets[0:5])

    else:
        text_proc = TextProcessor(metadata_path, input_paths, output_path, model="topic")
        corpus = text_proc.process_tweets(incident)

    return corpus

In [47]:
input_paths = ["/Users/stellachoi/Box Sync/research_work/events_tweets/Event - 319 - Moore Tornado/geotagged_from_archive/",
               "/Users/stellachoi/Box Sync/research_work/events_tweets/Event - 319 - Moore Tornado/user_timelines/"]
incident_metadata_path = '/Users/stellachoi/Box Sync/research_work/data_helper/incident_metadata.csv'
output_path = "/Users/stellachoi/Box Sync/research_work/disaster_social/twitter_analysis/data/topic_models/topics"
incident = 319
corpus = extract_tweet_with_period(incident, input_paths, output_path, incident_metadata_path)
tweets = corpus[0][:]
tweets.extend(corpus[1])
print("tweet_bf_len: {} tweet_af_len: {}".format(len(corpus[0]), len(corpus[1])))

tweet_bf_len: 435723 tweet_af_len: 769034


In [48]:
model_file = "topic_pipeline_80_v1.p"
[pipeline, model, lda, vect] = load_model(model_file, tweets)
[result_t, result_bf, result_af] = compare_topics(pipeline, model, len(corpus[0]))
show_topics(result_t, result_bf, result_af)


Topic Distribution Similarity 0.998817402296
Common topic [9, 77, 54, 14, 31, 49, 60, 15, 91, 87, 87, 75, 36, 110, 126, 62, 44, 58, 120, 76]
Unique Topic [71, 72, 78, 50]
BF unique topic [78, 72] ['0.0138394611571', '0.013490236918']
AF unique topic [71, 50] ['0.0152494770759', '0.0144349169293']



overall topic distribution: 

top_topics: 
 9 - 0.0557796030198 | 77 - 0.0209345579432 | 54 - 0.0202340244292 | 14 - 0.0183452538387 | 31 - 0.0176151778441 | 48 - 0.0173181706451 | 12 - 0.0166534047036 | 11 - 0.0160254648332 | 17 - 0.0160063858677 | 80 - 0.0155978194833 | 70 - 0.0149808475599 | 75 - 0.014950018846 | 36 - 0.0148373465891 | 71 - 0.0143380987276 | 44 - 0.0138730537005 | 4 - 0.0137809811049 | 58 - 0.0137671531846 | 56 - 0.0135890429957 | 50 - 0.0134899808269 | 76 - 0.0134613169107
topic 9 top words: 
 rt, today, news, saturday, wtf, goodnight, 16, respect, ladies, anybody, felt, obama, getglue, gets, george
topic 77 top words: 
 need, look, game, things, summer, okay, weekend, 

In [49]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, vect.transform(corpus[0]), vect, sort_topics=False, mds='tsne')
panel

/Users/stellachoi/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
0      1.299800        1       1  1.336044 -0.684198
1      1.104541        1       2  1.181489 -0.610650
2      0.756705        1       3  1.264605 -0.596994
3      1.392447        1       4  1.163885 -0.665186
4      1.115547        1       5  1.335852 -0.639816
5      1.232981        1       6  1.338758 -0.647654
6      1.060032        1       7  1.309056 -0.604133
7      0.800144        1       8  1.264509 -0.593690
8      6.267872        1       9  1.248893 -0.784394
9      0.936255        1      10  1.256654 -0.577471
10     1.706516        1      11  1.279457 -0.732118
11     1.855230        1      12  1.263101 -0.749673
12     1.209773        1      13  1.338398 -0.664043
13     1.751064        1      14  1.262362 -0.749677
14     0.825568        1      15  1.263042 -0.584802
15     0.907500        1      16  1.262905 -0.586491
16     1.945991        1      17  1.172320 -0.700504
17     1.275071        1      18  1.166334 -0.639150
18     1.249634        1      19  1.285689 -0.716578
19     1.051306        1      20  1.181767 -0.613133
20     1.283969        1      21  1.204691 -0.573534
21     1.259975        1      22  1.167235 -0.632589
22     0.884010        1      23  1.260869 -0.581744
23     1.039923        1      24  1.190545 -0.604092
24     0.913105        1      25  1.247710 -0.571497
25     1.192701        1      26  1.335531 -0.681589
26     0.793823        1      27  1.264735 -0.594656
27     0.607702        1      28  1.263025 -0.609871
28     0.731688        1      29  1.264411 -0.600387
29     1.221875        1      30  1.336417 -0.631383
...         ...      ...     ...       ...       ...
50     1.023832        1      51  1.248674 -0.570220
51     1.181634        1      52  1.169636 -0.630596
52     0.860611        1      53  1.263253 -0.587191
53     2.102149        1      54  1.177679 -0.718460
54     0.807159        1      55  1.264665 -0.593480
55     1.475573        1      56  1.162549 -0.661273
56     1.112067        1      57  1.336122 -0.658744
57     1.341549        1      58  1.168300 -0.624381
58     0.855375        1      59  1.264306 -0.589416
59     0.934351        1      60  1.257937 -0.578178
60     0.926952        1      61  1.326005 -0.626548
61     0.739704        1      62  1.264348 -0.601177
62     1.252840        1      63  1.337764 -0.675190
63     1.135786        1      64  1.259627 -0.576217
64     0.890855        1      65  1.261096 -0.584218
65     0.815124        1      66  1.264636 -0.591441
66     0.781118        1      67  1.264526 -0.599511
67     1.217737        1      68  1.337032 -0.676168
68     0.959397        1      69  1.261155 -0.581798
69     1.512695        1      70  1.162921 -0.633728
70     1.276825        1      71  1.174209 -0.703493
71     1.366474        1      72  1.164440 -0.660550
72     1.184359        1      73  1.337549 -0.669930
73     0.940911        1      74  1.259113 -0.579020
74     1.501573        1      75  1.332372 -0.703740
75     1.423315        1      76  1.336497 -0.691288
76     2.332638        1      77  1.261751 -0.755780
77     1.460328        1      78  1.327873 -0.706864
78     0.735714        1      79  1.264193 -0.602513
79     1.671323        1      80  1.268305 -0.742729

[80 rows x 5 columns], topic_info=     Category          Freq             Term         Total  loglift  logprob
term                                                                        
712   Default  67492.000000               rt  67492.000000  30.0000  30.0000
492   Default  18698.000000             like  18698.000000  29.0000  29.0000
508   Default  15670.000000              lol  15670.000000  28.0000  28.0000
523   Default  14288.000000             love  14288.000000  27.0000  27.0000
203   Default  12380.000000              day  12380.000000  26.0000  26.0000
464   Default  11695.000000             know  11695.0000

In [50]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, vect.transform(corpus[1]), vect, sort_topics=False, mds='tsne')
panel

/Users/stellachoi/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
0      1.253002        1       1  1.336044 -0.684198
1      1.094511        1       2  1.181489 -0.610650
2      0.779251        1       3  1.264605 -0.596994
3      1.382757        1       4  1.163885 -0.665186
4      1.101406        1       5  1.335852 -0.639816
5      1.185433        1       6  1.338758 -0.647654
6      1.036131        1       7  1.309056 -0.604133
7      0.786798        1       8  1.264509 -0.593690
8      5.926222        1       9  1.248893 -0.784394
9      0.935254        1      10  1.256654 -0.577471
10     1.760133        1      11  1.279457 -0.732118
11     1.903859        1      12  1.263101 -0.749673
12     1.192307        1      13  1.338398 -0.664043
13     1.738960        1      14  1.262362 -0.749677
14     0.908332        1      15  1.263042 -0.584802
15     0.855525        1      16  1.262905 -0.586491
16     1.618501        1      17  1.172320 -0.700504
17     1.283249        1      18  1.166334 -0.639150
18     1.401819        1      19  1.285689 -0.716578
19     1.038336        1      20  1.181767 -0.613133
20     1.204464        1      21  1.204691 -0.573534
21     1.407641        1      22  1.167235 -0.632589
22     0.895948        1      23  1.260869 -0.581744
23     1.005185        1      24  1.190545 -0.604092
24     1.089149        1      25  1.247710 -0.571497
25     1.205036        1      26  1.335531 -0.681589
26     0.854677        1      27  1.264735 -0.594656
27     0.612822        1      28  1.263025 -0.609871
28     0.744070        1      29  1.264411 -0.600387
29     1.221424        1      30  1.336417 -0.631383
...         ...      ...     ...       ...       ...
50     1.097688        1      51  1.248674 -0.570220
51     1.212539        1      52  1.169636 -0.630596
52     0.855500        1      53  1.263253 -0.587191
53     2.047524        1      54  1.177679 -0.718460
54     0.866412        1      55  1.264665 -0.593480
55     1.486127        1      56  1.162549 -0.661273
56     1.087603        1      57  1.336122 -0.658744
57     1.316125        1      58  1.168300 -0.624381
58     0.851749        1      59  1.264306 -0.589416
59     0.928439        1      60  1.257937 -0.578178
60     0.927562        1      61  1.326005 -0.626548
61     0.726975        1      62  1.264348 -0.601177
62     1.289745        1      63  1.337764 -0.675190
63     1.020923        1      64  1.259627 -0.576217
64     0.854221        1      65  1.261096 -0.584218
65     0.849213        1      66  1.264636 -0.591441
66     0.737495        1      67  1.264526 -0.599511
67     1.209871        1      68  1.337032 -0.676168
68     0.897292        1      69  1.261155 -0.581798
69     1.458981        1      70  1.162921 -0.633728
70     1.657706        1      71  1.174209 -0.703493
71     1.323179        1      72  1.164440 -0.660550
72     1.169858        1      73  1.337549 -0.669930
73     0.925509        1      74  1.259113 -0.579020
74     1.500266        1      75  1.332372 -0.703740
75     1.441277        1      76  1.336497 -0.691288
76     2.249681        1      77  1.261751 -0.755780
77     1.313305        1      78  1.327873 -0.706864
78     0.725166        1      79  1.264193 -0.602513
79     1.601279        1      80  1.268305 -0.742729

[80 rows x 5 columns], topic_info=     Category           Freq             Term          Total  loglift  logprob
term                                                                          
712   Default  113099.000000               rt  113099.000000  30.0000  30.0000
492   Default   33047.000000             like   33047.000000  29.0000  29.0000
508   Default   27580.000000              lol   27580.000000  28.0000  28.0000
523   Default   24665.000000             love   24665.000000  27.0000  27.0000
464   Default   20583.000000             know   20583.000000  26.0000  26.0000
203   Default   19733.000000              